## Adapt existing .Vtu to the new format

#### Step 0: load a .vtu file as XML 

In [2]:
import xml.etree.ElementTree as ET
import numpy as np

In [20]:
#vtu_file = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline"
#            "/rework/alpha/13_balance_flip/flip_flip/timestep_000001.vtu")
#vtu_file = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm"
#            "/FW_block_aggregated/timesteps/f6w14/timestep_000149.vtu"
#            "")
vtu_file = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline/rework"
            "/alpha/26_ghosts/with_ghosts/timestep_000003.vtu"
            "")
tree = ET.parse(vtu_file)
root = tree.getroot()

In [21]:
root[3][0][0].tag

'PointData'

In [22]:
# display the structure of the xml tree
for child in root:
    print(child.tag, child.attrib)
print(r'###')
pdata = root.findall('.//PointData')[0]
pdata.tag, pdata.attrib
is_typed = False
for child in pdata:
    print(child.tag, child.attrib)
    if child.attrib['Name']=='type':
        is_typed=True
if not is_typed:
    print("\n.vtu file has no type! will not work correctly in the simulation!")

dumpdate {}
tape {}
trisurf {'nvtx': '502', 'npoly': '0', 'nmono': '0', 'compressed': 'false', 'seed': '1625418592'}
UnstructuredGrid {}
###
DataArray {'type': 'Int64', 'Name': 'vertices_idx', 'format': 'ascii'}
DataArray {'type': 'Int64', 'Name': 'type', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'bonding_strength', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'direct_force', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'adhesion_strength', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'spontaneous_deviator', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'curvature', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'second_curvature', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'bending_modulus', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'second_bending_modulus', 'format': 'ascii'}
DataArray {'type': 'Float64'

In [67]:
## check that ghost vertices maintain relative position
#indices = (0,2,3,4,5,6,7,8,9,10,11,12,13,14)
#xs, ys, zs = np.zeros((len(indices),3)),np.zeros((len(indices),3)),np.zeros((len(indices),3))
#numbered_vtu_file = lambda x: ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline/rework"
#            f"/alpha/26_ghosts/with_ghosts/timestep_{x:06}.vtu"
#            "")
#for i, j in enumerate(indices):
#    tree = ET.parse(numbered_vtu_file(j))
#    root = tree.getroot()
#
#    cord_stuff = root.findall('.//*[@Name="Koordinate tock"]')[0]
#    #type_stuff = root.findall('.//*[@Name="type"]')[0]
#    xs[i,0], ys[i,0], zs[i,0] = [float(x) for x in cord_stuff.text.strip().splitlines()[5].split()]
#    xs[i,1], ys[i,1], zs[i,1] = [float(x) for x in cord_stuff.text.strip().splitlines()[6].split()]
#    xs[i,2], ys[i,2], zs[i,2] = [float(x) for x in cord_stuff.text.strip().splitlines()[7].split()]
#    #type_stuff.text.strip().split()[5:8]
#xs, ys, zs
#diff = zs[:,0]-zs[:,2]
#diff-diff[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

#### Step 1: reconstruct type. Determied by spontaneous curvature

In [119]:
spon_curv = pdata.findall('.//*[@Name="spontaneous_curvature"]')[0]
spon_curv.attrib

{'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'}

In [120]:
# type 4: bare vertex. type 39: active vertex
def type_from_curvature(curvatures):
    type_array = np.zeros(curvatures.shape)
    type_array[curvatures>0]=39
    type_array[curvatures<=0]=4
    return type_array

In [121]:
#an internal function from ts_vtu_python.py: 
def _xml_to_str_list(node):
    r"""Take xml node, extract text array to list.

    Convert sequence to 1d array and lines to 2d arrays, e.g
     2 3\n4 5 6\n' -> [['2','3'],['4','5','6']]
    '\n 2 3 5' -> ['2','3','5']
    """
    all_text = node.text.strip()

    # if there are tabs in the middle, assume this is a 2d list
    if all_text.__contains__('\n'):
        return [x.split() for x in all_text.split('\n')]
    else:  # no tabs: 1d array
        return all_text.split()

curvatures_txt = _xml_to_str_list(spon_curv)

In [122]:
curvatures = list(map(float,curvatures_txt)) # convert to float
cs = np.array(curvatures)
print(cs[:10],"...")

[0.  0.  0.  0.  0.  0.  0.  0.  0.8 0. ] ...


In [123]:
ar_type = type_from_curvature(cs)
print(ar_type[:10],"...")

[ 4.  4.  4.  4.  4.  4.  4.  4. 39.  4.] ...


In [126]:
def txt_from_float_array(a):
    return ''.join(f'{n:.17e} ' for n in a)
def txt_from_int_array(a):
    return ''.join(f'{int(n)} ' for n in a)

In [127]:
type_txt = txt_from_int_array(ar_type)

In [128]:
from_c, to_c = 500, 1500
print(f"existing text [{from_c}:{to_c}]:")
print(spon_curv.text[from_c:to_c])
print(f'\ngenerated text: [{from_c}:{to_c}]')
print(type_txt[from_c:to_c]) # peek at data

existing text [500:1500]:
+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000

In [129]:
# instead of updating the curvature like the previous cases, we want to add a node

#jupyter: make sure we didn't run this cell already
if pdata.find(".//*[@Name='type']") is None:
    type_node = ET.SubElement(pdata,"DataArray")
    print("made node")
type_node.attrib = {"type": "Int64", "Name": "type", "format": "ascii"}
type_node.text = type_txt

made node


In [131]:
for child in pdata: print(child.attrib)
#pdata.remove(pdata[3])

{'type': 'Int64', 'Name': 'vertices_idx', 'format': 'ascii'}
{'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'}
{'type': 'Float64', 'Name': 'bending_energy', 'format': 'ascii'}
{'type': 'Int64', 'Name': 'type', 'format': 'ascii'}


In [132]:
#see that the change applied
root2=tree.getroot()
print(root2.findall('.//*[@Name="type"]')[0].text.split(' ')[10:30])

['39', '4', '4', '4', '4', '4', '39', '4', '4', '4', '4', '4', '4', '4', '4', '4', '39', '4', '4', '39']


#### Step 2: Now we need to reconstruct the rest of the information based on the type

nodes to add:
>bonding_strength (w)  
>direct_force (f)  
>adhesion_strength (ad_w)  
>spontaneous_deviator (d)  
>normal*  
>force*  
>director*  

I don't think we *really* need accurate vector quantities, since they are recalculated during the step: we are likely to get only a few steps with "bad" values.  
we sweep the normals in `mean_curvature_and_energy`, and the force (and one day, deviator) should be update during the steps according to them  
-initialize them at 0.

In [135]:
# get values from tape
tape_txt = root.findall(".//*[@Name='tape']")[0].text

IndexError: list index out of range

In [ ]:
# bonding
if pdata.find(".//*[@Name='bonding_strength']") is None:
    type_node = ET.SubElement(pdata,"DataArray")
    print("made node")
type_node.attrib = {"type": "Int64", "Name": "type", "format": "ascii"}

type_node.text = type_txt

In [42]:
#!!! uncomment to modify the vtu file!!!
#tree.write(r'/mnt/c/Users/yoavr/Desktop/paraview_pipeline/tests/round_100_neg.vtu',encoding=r'UTF-8')